In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  24.38 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  36.98 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  51.93 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  37.00 MB/s
vincent-0.4.4- 100% |###################

## Download Dataset

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
neighborhoods_data = newyork_data['features']

### Transform the data into pandas dataframe

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [6]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
    

In [7]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


### Form dataframe with Manhattan Neighborhoods

In [8]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.sort_values(by = 'Neighborhood', axis = 0, inplace = True)

manhattan_data.shape

(40, 4)

#### Define Foursquare Credentials and Version

In [9]:
CLIENT_ID = 'BXZPG3NUBOSGVJMFMDGMLMD1SYCNOTRGH5ZMAA1OY0C3HJ50' # your Foursquare ID
CLIENT_SECRET = 'YQFFK1MY1VF4MRKLHXZAP03VSXT1UNCYHXUKTR1H4NUPNDU2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BXZPG3NUBOSGVJMFMDGMLMD1SYCNOTRGH5ZMAA1OY0C3HJ50
CLIENT_SECRET:YQFFK1MY1VF4MRKLHXZAP03VSXT1UNCYHXUKTR1H4NUPNDU2


In [10]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


## Search for Yoga centers and Gyms in Manhattan area

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        LIMIT = 100 # limit of number of venues returned by Foursquare API
        radius = 500 # define radiu
        search_query = 'Gym','yoga'
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            lat, 
            lng,
            VERSION,
            search_query,
            radius, 
            LIMIT)

        results = requests.get(url).json()["response"]['venues']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'],
            v['location']['formattedAddress'][-2],
            v['location']['distance'],
            v['location']['lat'], 
            v['location']['lng'],
            ) for v in results])

    nearby_venues = pd.DataFrame([item for gym_venues in venues_list for item in gym_venues])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'PostalCode',
                  'distance',
                  'Venue Latitude', 
                  'Venue Longitude',
                  ]      
    
    return(nearby_venues)


#### Code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [12]:

manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )


Battery Park City
Carnegie Hill
Central Harlem
Chelsea
Chinatown
Civic Center
Clinton
East Harlem
East Village
Financial District
Flatiron
Gramercy
Greenwich Village
Hamilton Heights
Hudson Yards
Inwood
Lenox Hill
Lincoln Square
Little Italy
Lower East Side
Manhattan Valley
Manhattanville
Marble Hill
Midtown
Midtown South
Morningside Heights
Murray Hill
Noho
Roosevelt Island
Soho
Stuyvesant Town
Sutton Place
Tribeca
Tudor City
Turtle Bay
Upper East Side
Upper West Side
Washington Heights
West Village
Yorkville


In [13]:
manhattan_venues['PostalCode'] = manhattan_venues['PostalCode'].str[-5:]
manhattan_venues = manhattan_venues[~manhattan_venues.PostalCode.str.contains('NY')]
manhattan_venues = manhattan_venues[~manhattan_venues.PostalCode.str.contains('York')]
manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,PostalCode,distance,Venue Latitude,Venue Longitude
0,Battery Park City,40.711932,-74.016869,225 Rector Place Gym and Studio,10280,328,40.709183,-74.015441
1,Battery Park City,40.711932,-74.016869,The Gym At Soundings,10280,368,40.708625,-74.016795
2,Battery Park City,40.711932,-74.016869,Stuyvesant High School (3rd floor gym),10282,196,40.710186,-74.016563
3,Battery Park City,40.711932,-74.016869,Gym at WFC 2,10280,132,40.712924,-74.016003
4,Battery Park City,40.711932,-74.016869,Gym @ St. John's,10007,513,40.715166,-74.012526


In [14]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,PostalCode,distance,Venue Latitude,Venue Longitude
Neighborhood,,,,,,,
Battery Park City,32,32,32,32,32,32,32
Carnegie Hill,25,25,25,25,25,25,25
Central Harlem,5,5,5,5,5,5,5
Chelsea,34,34,34,34,34,34,34
Chinatown,14,14,14,14,14,14,14
Civic Center,42,42,42,42,42,42,42
Clinton,44,44,44,44,44,44,44
East Harlem,7,7,7,7,7,7,7
East Village,20,20,20,20,20,20,20


# Prepare for K-means

In [31]:
manhattan_venues_new = pd.DataFrame([manhattan_venues.PostalCode, manhattan_venues.distance]).transpose()
manhattan_venues_new.head()

,PostalCode,distance
0,10280,328
1,10280,368
2,10282,196
3,10280,132
4,10007,513


In [16]:
print('There are {} fitness centers in the neighborhood of Manhattan.'.format(len(manhattan_venues['Venue'].unique())))

There are 752 fitness centers in the neighborhood of Manhattan.


### Separate out Gyms in Manhattan Neighborhoods

In [17]:
search_query = 'Gym'
radius = 10000
LIMIT = 100
print(search_query + ' .... OK!')

Gym .... OK!


In [18]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=BXZPG3NUBOSGVJMFMDGMLMD1SYCNOTRGH5ZMAA1OY0C3HJ50&client_secret=YQFFK1MY1VF4MRKLHXZAP03VSXT1UNCYHXUKTR1H4NUPNDU2&ll=40.7900869,-73.9598295&v=20180605&query=Gym&radius=10000&limit=100'

In [19]:
results = requests.get(url).json()
results
# assign relevant part of JSON to venues
gym_venues = results['response']['venues']

# tranform venues into a dataframe
gym_df = json_normalize(gym_venues)


In [20]:
# keep only columns that include venue name, and anything that is associated with location
gym_filtered_columns = ['name', 'categories'] + [col for col in gym_df.columns if col.startswith('location.')] + ['id']
gym_df_filtered = gym_df.loc[:, gym_filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
gym_df_filtered['categories'] = gym_df_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
gym_df_filtered.columns = [column.split('.')[-1] for column in gym_df_filtered.columns]

gym_df_filtered


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Gym at 808 Columbus,Gym,808 Columbus Ave,US,New York,United States,at 98th St,773,"[808 Columbus Ave (at 98th St), New York, NY 1...","[{'lng': -73.96613557327645, 'label': 'display...",40.795141,-73.966136,10025,NY,4c18446eed98ef3b9e8edd02
1,Mid-City Gym & Tanning,Gym,345 W 42nd St,US,New York,United States,between 8th & 9th Avenues,4461,"[345 W 42nd St (between 8th & 9th Avenues), Ne...","[{'lng': -73.9917226956553, 'label': 'display'...",40.758109,-73.991723,10036,NY,4a60c4d0f964a5202bc11fe3
2,The Dalton School Gym,College Gym,200 E 87th St,US,New York,United States,btwn 2nd & 3rd Ave,1329,"[200 E 87th St (btwn 2nd & 3rd Ave), New York,...","[{'lng': -73.95325541496277, 'label': 'display...",40.779233,-73.953255,10128,NY,4adb1afcf964a520642421e3
3,The Little Gym of The Upper West Side,Gym,2121 Broadway,US,New York,United States,74th Street,2097,"[2121 Broadway (74th Street), New York, NY 100...","[{'lng': -73.98133382906045, 'label': 'display...",40.780607,-73.981334,10023,NY,4c01877ff7ab0f47e54b16b6
4,Levien Gym - Columbia University,College Basketball Court,3030 Broadway,US,New York,United States,W 119th St & Broadway,2215,"[3030 Broadway (W 119th St & Broadway), New Yo...","[{'lng': -73.96241621335183, 'label': 'display...",40.809889,-73.962416,10027,NY,4ce1ccc5c4f6a35de151dd6c
5,180 East End Avenue Gym,Gym,180 E End Ave,US,New York,United States,btwn 89th and 90th,2021,"[180 E End Ave (btwn 89th and 90th), New York,...","[{'lng': -73.94406427364363, 'label': 'display...",40.776405,-73.944064,10128,NY,4d07a00428926ea8115f7bc2
6,Gym @ One UN New York,Gym,"Manhattan, NY",US,New York,United States,NaN,23,"[Manhattan, NY, New York, NY, United States]","[{'lng': -73.9597222, 'label': 'display', 'lat...",40.790278,-73.959722,NaN,NY,5142e617e4b0041cbc8e2a6d
7,GYM Sportsbar,Gay Bar,167 8th Ave,US,New York,United States,btwn W 18th & W 19th St,6309,"[167 8th Ave (btwn W 18th & W 19th St), New Yo...","[{'lng': -74.00057042515249, 'label': 'display...",40.742539,-74.000570,10011,NY,42829c80f964a52032221fe3
8,Central Rock Gym,Climbing Gym,21 W End Ave,US,New York,United States,60th & West End Ave,3235,"[21 W End Ave (60th & West End Ave), New York,...","[{'lng': -73.990531, 'label': 'display', 'lat'...",40.772645,-73.990531,10023,NY,5ac8f751911fc41079563f77
9,Gym,Gym,1 Morningside Dr,US,New York,United States,Avalon Morningside Park,1343,"[1 Morningside Dr (Avalon Morningside Park), N...","[{'lng': -73.96129720428054, 'label': 'display...",40.802107,-73.961297,10025,NY,4afc1d83f964a520ef1f22e3


### Separate out Yoga centers in Manhattan Neighborhoods

In [21]:
search_query = 'yoga'
radius = 10000
LIMIT = 100
print(search_query + ' .... OK!')

yoga .... OK!


In [22]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=BXZPG3NUBOSGVJMFMDGMLMD1SYCNOTRGH5ZMAA1OY0C3HJ50&client_secret=YQFFK1MY1VF4MRKLHXZAP03VSXT1UNCYHXUKTR1H4NUPNDU2&ll=40.7900869,-73.9598295&v=20180605&query=yoga&radius=10000&limit=100'

In [23]:
results = requests.get(url).json()
results
# assign relevant part of JSON to venues
yoga_venues = results['response']['venues']

# tranform venues into a dataframe
yoga_df = json_normalize(yoga_venues)


In [24]:
# keep only columns that include venue name, and anything that is associated with location
yoga_filtered_columns = ['name', 'categories'] + [col for col in yoga_df.columns if col.startswith('location.')] + ['id']
yoga_df_filtered = yoga_df.loc[:, yoga_filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
yoga_df_filtered['categories'] = yoga_df_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
yoga_df_filtered.columns = [column.split('.')[-1] for column in yoga_df_filtered.columns]

yoga_df_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Pure Yoga East,Yoga Studio,203 E 86th St,US,New York,United States,at 3rd Ave,1364,"[203 E 86th St (at 3rd Ave), New York, NY 1002...","[{'lng': -73.95377186715457, 'label': 'display...",40.778723,-73.953772,NaN,10028,NY,4a049afcf964a5200f721fe3
1,Upper West Side Yoga and Wellness,Yoga Studio,75 W 85th St Apt B,US,New York,United States,btwn Columbus Ave. & Central Park West,1160,[75 W 85th St Apt B (btwn Columbus Ave. & Cent...,"[{'lng': -73.97231017100543, 'label': 'display...",40.785682,-73.972310,NaN,10024,NY,516827db72da218d58fdf80b
2,Pure Yoga West,Yoga Studio,204 W 77th St,US,New York,United States,at Amsterdam Ave,1908,"[204 W 77th St (at Amsterdam Ave), New York, N...","[{'lng': -73.97954526697957, 'label': 'display...",40.781659,-73.979545,NaN,10024,NY,4b365ac3f964a520773325e3
3,New York Yoga - Hot Studio,Yoga Studio,132 E 85th St,US,New York,United States,Lexington Ave,1282,"[132 E 85th St (Lexington Ave), New York, NY 1...","[{'lng': -73.95615953339332, 'label': 'display...",40.778910,-73.956160,NaN,10028,NY,4ad7bdb0f964a5204d0e21e3
4,Life in Motion: Yoga Cycling Barre,Yoga Studio,2744 Broadway,US,New York,United States,@ 105th Street,1345,"[2744 Broadway (@ 105th Street), New York, NY ...","[{'lng': -73.96791041268004, 'label': 'display...",40.800515,-73.967910,NaN,10025,NY,4a86b49cf964a5201d0120e3


## Visualize the gyms & Yoga centers in Manhattan area

In [25]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel


# add the gyms as blue circle markers
for lat, lng, label in zip(gym_df_filtered.lat, gym_df_filtered.lng, gym_df_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
    
    
# add the yoga centers as green circle markers
for lat, lng, label in zip(yoga_df_filtered.lat, yoga_df_filtered.lng, yoga_df_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(venues_map)
    
# add the Manhattan neighborhoods as yellow circle markers
for lat, lng, label in zip(manhattan_data.Latitude, manhattan_data.Longitude, manhattan_data.Neighborhood):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='yellow',
        popup=label,
        fill = True,
        fill_color='yellow',
        fill_opacity=0.6
    ).add_to(venues_map)
# display map
venues_map

### Cluster Neighborhoods

In [32]:
# set number of clusters
kclusters = 5


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_venues_new)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [36]:
manhattan_venues.insert(0, 'Cluster Labels', kmeans.labels_)
manhattan_venues.head()

,Cluster Labels,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,PostalCode,distance,Venue Latitude,Venue Longitude
0,0,Battery Park City,40.711932,-74.016869,225 Rector Place Gym and Studio,10280,328,40.709183,-74.015441
1,0,Battery Park City,40.711932,-74.016869,The Gym At Soundings,10280,368,40.708625,-74.016795
2,0,Battery Park City,40.711932,-74.016869,Stuyvesant High School (3rd floor gym),10282,196,40.710186,-74.016563
3,0,Battery Park City,40.711932,-74.016869,Gym at WFC 2,10280,132,40.712924,-74.016003
4,0,Battery Park City,40.711932,-74.016869,Gym @ St. John's,10007,513,40.715166,-74.012526


### Visualize the resulting clusters

In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_venues['Venue Latitude'], manhattan_venues['Venue Longitude'], manhattan_venues['Neighborhood'], manhattan_venues['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine the clusters

In [38]:
manhattan_venues.loc[manhattan_venues['Cluster Labels'] == 0, manhattan_venues.columns[[1] + list(range(5, manhattan_venues.shape[1]))]]

,Neighborhood,PostalCode,distance,Venue Latitude,Venue Longitude
0,Battery Park City,10280,328,40.709183,-74.015441
1,Battery Park City,10280,368,40.708625,-74.016795
2,Battery Park City,10282,196,40.710186,-74.016563
3,Battery Park City,10280,132,40.712924,-74.016003
4,Battery Park City,10007,513,40.715166,-74.012526
5,Battery Park City,10280,350,40.708995,-74.018358
6,Battery Park City,10282,380,40.715301,-74.016120
7,Battery Park City,10280,187,40.711606,-74.014689
8,Battery Park City,10280,210,40.710231,-74.017948
10,Battery Park City,10280,108,40.711350,-74.017896


In [39]:
manhattan_venues.loc[manhattan_venues['Cluster Labels'] == 1, manhattan_venues.columns[[1] + list(range(5, manhattan_venues.shape[1]))]]

,Neighborhood,PostalCode,distance,Venue Latitude,Venue Longitude
443,Greenwich Village,60051,480,40.729931,-74.004014
1158,West Village,60051,533,40.729931,-74.004014


In [40]:
manhattan_venues.loc[manhattan_venues['Cluster Labels'] == 2, manhattan_venues.columns[[1] + list(range(5, manhattan_venues.shape[1]))]]

,Neighborhood,PostalCode,distance,Venue Latitude,Venue Longitude
121,Chelsea,77008,371,40.7456,-74.007


In [41]:
manhattan_venues.loc[manhattan_venues['Cluster Labels'] == 3, manhattan_venues.columns[[1] + list(range(5, manhattan_venues.shape[1]))]]

,Neighborhood,PostalCode,distance,Venue Latitude,Venue Longitude
628,Little Italy,23219,366,40.71975,-74.001614


In [42]:
manhattan_venues.loc[manhattan_venues['Cluster Labels'] == 4, manhattan_venues.columns[[1] + list(range(5, manhattan_venues.shape[1]))]]

,Neighborhood,PostalCode,distance,Venue Latitude,Venue Longitude
697,Midtown,-4012,316,40.756635,-73.9844
